In [1]:
import sys
import os
from pathlib import Path
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append(str(Path.cwd().parent))
from config import PATH_KIOSK_USER_PATTERNS_FOLDER, PATH_KIOSK_USER_PATTERNS_REPO, PATH_DATA_SHARING_FOLDER

# Example usage
kupdat03_path = PATH_DATA_SHARING_FOLDER /'DEWEY'
execfile(PATH_KIOSK_USER_PATTERNS_REPO / 'constants.py')
execfile(PATH_KIOSK_USER_PATTERNS_REPO / 'functions/extract_file_list.py')


In [2]:
import numpy as np
import deweydatapy as ddp

# Create date range from Jan 1, 2019 to June 30, 2025
dates = pd.date_range(start='2019-01-01', end='2025-06-30', freq='D')
df_dates = pd.DataFrame({'date': dates})
df_dates['is_weekday'] = df_dates['date'].dt.weekday < 5

# Add year and month columns for grouping
df_dates['year'] = df_dates['date'].dt.year
df_dates['month'] = df_dates['date'].dt.month

# Function to sample one weekday and one weekend per group
def sample_days(group):
    weekday = group[group['is_weekday']].sample(1, random_state=42) if group['is_weekday'].any() else pd.DataFrame()
    weekend = group[~group['is_weekday']].sample(1, random_state=42) if (~group['is_weekday']).any() else pd.DataFrame()
    return pd.concat([weekday, weekend])

# Apply sampling per year-month group
sampled = df_dates.groupby(['year', 'month'], group_keys=False).apply(sample_days).reset_index(drop=True)

sampled.head()

C:\Users\jvargh7\AppData\Local\Temp\ipykernel_16288\414929021.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled = df_dates.groupby(['year', 'month'], group_keys=False).apply(sample_days).reset_index(drop=True)


,date,is_weekday,year,month
0,2019-01-22,True,2019,1
1,2019-01-06,False,2019,1
2,2019-02-01,True,2019,2
3,2019-02-03,False,2019,2
4,2019-03-01,True,2019,3


In [3]:
# data_endpoint = "https://app.deweydata.io/external-api/v3/products/c117bbd0-ed3a-4c05-8f3c-01ac4b12ef60/files"
data_endpoint = "https://app.deweydata.io/api/v1/external/data/fldr_8zme9bwbekydvezq"


In [ ]:
# m = 12 # pending
for y in range(2024, 2018, -1):
    for m in range(11,0,-1):
        start = f"{y}-{m:02d}-01"
        if m == 2:
            end = f"{y}-{m:02d}-28" # doesn't account for leap years, but we are not going that far back
        elif m in [4, 6, 9, 11]:
            end = f"{y}-{m:02d}-30"
        else:
            end = f"{y}-{m:02d}-31"
        if pd.to_datetime(end) > pd.to_datetime('2025-05-31'):
            end = '2025-05-31'

        print(start + " to " + end)
        files, meta = extract_file_list(bulk_api_veraset_visits, data_endpoint, sd=start, ed=end)
        files.to_csv(PATH_KIOSK_USER_PATTERNS_REPO / 'data' / f'kupdat03b_veraset_visits_{y}_{m:02d}.csv', index=False)

        # Ensure 'partition_key' in files and 'date' in sampled are both datetime type
        files['partition_key_dt'] = pd.to_datetime(files['partition_key'])
        sampled['date_dt'] = pd.to_datetime(sampled['date'])

        # Perform inner join on 'partition_key' and 'date'
        merged = pd.merge(files, sampled, left_on='partition_key_dt', right_on='date_dt', how='inner')
        merged.to_csv(PATH_KIOSK_USER_PATTERNS_REPO / 'data' / f'kupdat03b_veraset visits after sampling_{y}_{m:02d}.csv', index=False)


        dest_dir = fr'R:/Global Diabetes Research Center/Patel/Veraset/Sampled Visit Dates/{y}/'
        os.makedirs(dest_dir, exist_ok=True)


        ddp.download_files(merged, dest_folder = dest_dir,   skip_exists= True)

2024-12-01 to 2024-12-31
 
Metadata summary ------------------------------------------------
Total number of files: 240,709
Total files size (MB): 11,456,926.18
Date aggregation: DAY
Date partition column: UTC_TIMESTAMP
Data min available date: 2019-01-01
Data max available date: 2025-05-31
-----------------------------------------------------------------
 
Files information collection completed.
 
Files information summary ---------------------------------------
Total number of pages: 154
Total number of files: 7,680
Total files size (MB): 228,078.97
Average single file size (MB): 29.69
Date partition column: UTC_TIMESTAMP
Expires at: 2025-07-14T21:19:26.292635+00:00
-----------------------------------------------------------------
Writing R:/Global Diabetes Research Center/Patel/Veraset/Sampled Visit Dates/2024/part-00000-tid-3900123142972219622-a0b40fad-b26d-4831-ac7c-5f67350cf951-1149-1-c000.snappy.parquet
Please be patient. It may take a while...
   
Writing R:/Global Diabetes Res

KeyboardInterrupt: 